# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
from datetime import datetime

%load_ext sql
%load_ext dotenv
%config SqlMagic.displaylimit = 5

In [ ]:
from dotenv import find_dotenv
from dotenv import load_dotenv

load_dotenv(find_dotenv())

In [ ]:
%dotenv -o

### DB Connection

In [ ]:
import google.auth
from google.cloud.sql.connector import Connector
import sqlalchemy

DB_USER = os.getenv("DATABASE_USERNAME", "")
DB_NAME = os.getenv("DATABASE_NAME", "")
DB_PASSWORD = os.getenv("DATABASE_PASSWORD", "")
DB_HOST = os.getenv("DATABASE_HOST", "")
DB_PORT = os.getenv("DATABASE_PORT", "")
DB_CONNECTION_NAME = os.getenv("LEAR_DB_CONNECTION_NAME")  # project:region:instance-name

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        DB_CONNECTION_NAME,
        "pg8000",
        ip_type="public",
        user=DB_USER,
        db=DB_NAME,
        enable_iam_auth=True
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    if not DB_CONNECTION_NAME:
        connect_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
        pool = sqlalchemy.create_engine(connect_string)
    else:
        # NOTE: also need GOOGLE_APPLICATION_CREDENTIALS to be set in the .env
        #    - when running the notebook on its own, the ^ cred path starts in this folder instead of the root of the service
        #    - if this path is invalid or the creds are invalid the line below will fail
        credentials, project_id = google.auth.default()
        pool = sqlalchemy.create_engine(
            "postgresql+pg8000://",
            creator=get_conn,
        )
    
    return pool

In [ ]:
lear_pool = get_pool()
%sql lear_pool --alias lear
%sql lear

In [ ]:
%sql --connections

Simple query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b 
    ,businesses_version old 
    ,businesses_version new 
where f.filing_type in ('alteration', 'changeOfName')   
    and f.business_id=b.id
    and b.legal_type in ('CP', 'XCP')
    and f.transaction_id=old.end_transaction_id 
    and f.transaction_id=new.transaction_id
    and f.source='LEAR' 
    and old.legal_name != new.legal_name
    and f.id not in (select filing_id from sent_to_gazette);  

In [ ]:
%%sql  coop_gazette_changeofname  <<
select
  to_char(f.effective_date,'MON dd, yyyy')
||';'
||rpad(b.identifier,10)
||';'
||rpad(substr(old.legal_name,1,52),52)
||';'
||' to '
||';'
||rpad(substr(new.legal_name,1,58),58)
from filings      f
    ,businesses   b   
    ,sent_to_gazette stg
    ,businesses_version old 
    ,businesses_version new 
where f.filing_type in ('alteration', 'changeOfName')
    and f.business_id=b.id
    and b.legal_type in ('CP', 'XCP')
    and f.id=stg.filing_id  
    and f.transaction_id=old.end_transaction_id 
    and f.transaction_id=new.transaction_id
    and old.legal_name != new.legal_name
    and stg.sent_to_gazette_date is null   
order by f.effective_date;

In [ ]:
datestr = datetime.strftime(datetime.now(), '%Y%m%d')
coop_gazette_changeofname_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_CHANGEOFNAME_' + datestr +'.TXT'

with open(coop_gazette_changeofname_filename, 'w') as f:
    if not coop_gazette_changeofname.DataFrame().empty:
        dfAsString = coop_gazette_changeofname.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b    
where f.filing_type in ('dissolved', 'voluntaryLiquidation', 'dissolution') 
    and f.business_id=b.id 
    and b.legal_type in ('CP', 'XCP')
    and f.source='LEAR' 
    and f.id not in (select filing_id from sent_to_gazette);

In [ ]:
%%sql coop_gazette_dissolution  <<
select
  to_char(f.effective_date,'yyyymmdd')
||';'
||rpad(b.identifier,10)                  
||';' 
||rpad(b.legal_name,150)
||';'
|| CASE WHEN f.filing_type = 'dissolved' THEN '1'
        WHEN f.filing_type = 'voluntaryLiquidation' THEN '2'
        WHEN f.filing_type = 'dissolution' THEN '3'        
   end   
||rpad(' ',27)
from filings      f
    ,businesses   b   
    ,sent_to_gazette stg
where f.filing_type in ('dissolved', 'voluntaryLiquidation', 'dissolution')  
      and f.business_id=b.id
      and f.id=stg.filing_id      
      and b.legal_type in ('CP', 'XCP')
      and stg.sent_to_gazette_date is null   
order by f.effective_date;      

In [ ]:
coop_gazette_dissolution_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_DISSOLUTION_' + datestr +'.TXT'

with open(coop_gazette_dissolution_filename, 'w') as f:  
    if not coop_gazette_dissolution.DataFrame().empty:
        dfAsString = coop_gazette_dissolution.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b    
where f.filing_type in ('incorporationApplication','amalgamationApplication') 
    and f.business_id=b.id 
    and b.legal_type in ('CP', 'XCP')
    and f.source='LEAR' 
    and f.id not in (select filing_id from sent_to_gazette);

In [ ]:
%%sql coop_gazette_incorporation  <<
select
  to_char(b.founding_date,'MON dd, yyyy')
||';'
||rpad(b.identifier,10)              
||';'
||rpad(b.legal_name,150)
||rpad(' ',59)
from filings      f
    ,businesses   b    
    ,sent_to_gazette stg
where f.filing_type in ('incorporationApplication','amalgamationApplication') 
    and f.business_id=b.id 
    and f.id=stg.filing_id
    and b.legal_type in ('CP', 'XCP')
    and stg.sent_to_gazette_date is null   
order by b.founding_date;

In [ ]:
coop_gazette_incorporation_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_INCORPORATION_' + datestr +'.TXT'

with open(coop_gazette_incorporation_filename, 'w') as f:
    if not coop_gazette_incorporation.DataFrame().empty:
        dfAsString = coop_gazette_incorporation.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)

In [ ]:
%%sql 
insert into sent_to_gazette (filing_id, identifier, sent_to_gazette_date)
select f.id, b.identifier, null
from filings      f
    ,businesses   b    
where f.filing_type = 'restoration' 
    and f.business_id=b.id 
    and b.legal_type in ('CP', 'XCP')
    and f.source='LEAR' 
    and f.id not in (select filing_id from sent_to_gazette);

In [ ]:
%%sql  coop_gazette_restoration  <<
select
  to_char(f.effective_date,'yyyymmdd')
||';'
||rpad(b.identifier,10)        
||';'
||rpad(b.legal_name,112)
from filings      f
    ,businesses   b    
    ,sent_to_gazette stg
where f.filing_type = 'restoration'  
  and f.business_id=b.id 
  and f.id=stg.filing_id
  and b.legal_type in ('CP', 'XCP')
  and stg.sent_to_gazette_date is null 
order by f.effective_date;

In [ ]:
coop_gazette_restoration_filename = os.getenv('DATA_DIR', '')+'COOP_GAZETTE_RESTORATION_' + datestr +'.TXT'

with open(coop_gazette_restoration_filename, 'w') as f:
    if not coop_gazette_restoration.DataFrame().empty:
        dfAsString = coop_gazette_restoration.DataFrame().to_string(header=False, index=False)
        f.write(dfAsString)